# 텔레그램 봇 기능 확장하기

* 난이도 : ★★★★☆☆☆☆☆☆
* 필요라이브러리: telepot, logging, requests, BeautifulSoup4


* 크롤링을 활용하여 텔레그램 봇에 여러가지 기능을 추가해보도록 하겠습니다.
* 추가할 기능 요약:

<table width="100%">
    <tr>
        <td width=200><p>명령어 예제</p></td>
        <td width=400><p>기능</p></td>
    </tr>
    <tr>
        <td><p>/dir C:\test</span></td>
        <td><p>경로의 폴더 및 파일목록 전송</p></td>
    </tr>
    <tr>
        <td><p align='left'>/getfile C:\test\문서.txt</p></td>
        <td><p align='left'>경로에 있는 파일을 전송</p></td>
    </tr>
    <tr>
        <td><p align='left'>/getimage C:\test\test.jpg</p></td>
        <td><p align='left'>경로에 있는 이미지파일을 전송</p></td>
    </tr>
    <tr>
        <td><p align='left'>/getaudio C:\test\test.mp3</p></td>
        <td><p align='left'>경로에 있는 오디오파일을 전송</p></td>
    </tr>
    <tr>
        <td><p align='left'>/getvideo C:\test\test.mp4</p></td>
        <td><p align='left'>경로에 있는 비디오파일을 전송</p></td>
    </tr>
    <tr>
        <td><p align='left'>/weather 서울</p></td>
        <td><p align='left'>날씨를 구해 알려주는 기능을 추가</p></td>
    </tr>
    <tr>
        <td><p align='left'>/money 150 달라</p></td>
        <td><p align='left'>통화명에 맞는 환율을 구해서 알려주는 기능</p></td>
    </tr>
</table>


## modules.py 파일에 추가 기능 작성

* <b style='color:red'>코드의 가독성을 위해 추가 기능과 텔레그램 봇 파일을 분리하겠습니다.</b>
* 텔레그램 봇 기능은 (telegram.py) 로 하고 기능 파일은 (modules.py) 로 하겠습니다.
* 모듈의 기능은 텔레그램 봇 파일(telegram.py) 에서 **from modules import *** 을 선언해서 사용할 수 있습니다.

### /weather 명령어에 대한 기능 작성

![bot_7.jpg](images/bot_7.jpg)


* 날씨 데이터는 네이버 검색 결과를 활동하도록 합니다.

In [ ]:
'''
modules.py 파일
'''

import requests
from bs4 import BeautifulSoup

def get_weather(where):
    '''네이버에서 날씨를 검색해 파싱하는 함수

    Args:
        where (str) : 지역명
    
    Returns:
        str : 결과 문자열
    '''
    
    # 네이버 검색어를 서울+날씨 이런식으로 검색어에 '날씨' 단어를 조합해서 쿼리합니다.
    url = "https://search.naver.com/search.naver?query={}+날씨".format(where)
    r = requests.get(url)
    bs = BeautifulSoup(r.text, "lxml")
    
    # 개발자도구를 활용하여 날씨 데이터의 최상위 요소를 확인합니다.
    # 이는 날씨 결과가 존재하는지 아닌지를 판단하기 위한 근거로 사용됩니다.
    weather_box = bs.select("div.weather_box")

    # 최종 결과를 리턴할 문자열 변수
    weather = ""

    # 날씨 결과가 존재 한다면
    if len(weather_box) > 0:
        # 실제 날짜 데이터의 온도, 미세먼지 등의 데이터를 파싱 합니다.
        temperature = bs.select("div.main_info > div.info_data > p.info_temperature > span.todaytemp")
        indecator = bs.select("dl.indicator")
        txt = bs.select("div.today_area._mainTabContent > div.main_info > div.info_data > ul.info_list > li > p.cast_txt")

        # 파싱한 데이터의 유효성은 파싱된 데이터의 갯수로 체크합니다.
        if len(temperature) > 0 and len(indecator) > 0 and len(txt) > 0:

            # 뷰티풀숩으로 select 된 요소는 리스트 형태기 때문에
            # 각요소의[0]번째 내용을 변수화 시킵니다.
            temperature = temperature[0].text.strip()
            indecator = indecator[0].text.strip()
            txt = txt[0].text.strip()

            # 하나의 문자열 데이터로 만듭니다.
            weather = "{}℃\r\n{}\r\n{}".format(temperature, indecator, txt)
            
    # 날씨 문자열을 리턴합니다.
    return weather
    
print(get_weather("서울"))

### /money 명령어에 대한 기능 추가
* ***/money 15 달라*** 이런식으로 명령어를 날리면 ***17,550원*** 이런 결과를 리턴되게 하고 싶습니다.
* 그럼 먼저 각 통화에 대한 환율을 알아야 합니다.
* 환율 정보는 네이버 환율 정보를 크롤링 하여 사용하도록 합니다.
* 네이버 금융(https://finance.naver.com/marketindex/)의 시장지표를 보면 환율 목록이 나오는데 이 페이지는 iframe 형태로 되어있습니다. iframe 은 페이지 안에 다른 페이지를 보여주는 HTML 태그 입니다. 우리가 필요한 페이지는 iframe 으로 되어있는 페이지입니다.
* https://finance.naver.com/marketindex/exchangeList.nhn 주소의 데이터를 사용하도록 하겠습니다.

In [ ]:
'''
modules.py 파일
'''

import requests
from bs4 import BeautifulSoup

def get_exchange_info():
    '''네이버 금융의 iframe 안의 환율정보 페이지에서 각 통화의 환율을 구해서 저장하는 함수'''
    
    # 최종 각 통화에 대한 환율 정보를 저장하여 리턴할 딕셔너리 변수
    EXCHANGE_LIST = {}
    
    # 네이버 금융의 iframe 안에 있는 환율 정보 페이지 주소 URL
    url = "https://finance.naver.com/marketindex/exchangeList.nhn"
    r = requests.get(url)
    bs = BeautifulSoup(r.content, "lxml")
    
    # 환율 정보 파싱
    trs = bs.select("table.tbl_exchange > tbody > tr") # list 자료형을 리턴함
    
    for tr in trs:
        tds = tr.select("td")
        # 테이블 td 의 컬럼이 7개면 정상적인 데이터라고 판단
        if len(tds) == 7:
            # 통화명
            name = tds[0].text.strip() # 미국 USD
            # 환율정보
            value = float(tds[1].text.strip().replace(",", "")) # 1,120
            # 최종 리턴될 딕셔너리 변수에 통화명을 키로 환율정보를 값으로 저장
            EXCHANGE_LIST[name] = value
    # 최종 결과 리턴
    return EXCHANGE_LIST


'''
사용자가 입력한 통화명을 네이버 금융의 환율 데이터의 이름과 매칭 시키기 위한 데이터
예) 150 달라 를 입력했을때 '달라'의 이름을 '미국 USD' 로 매칭해야 환율을 구하기가 쉬워서
'''
MONEY_NAME = {
    "달라": "미국 USD",
    "유로": "유럽연합 EUR",
    "엔": "일본 JPY (100엔)",
    "위안": "중국 CNY",
    "홍콩달라": "홍콩 HKD",
    "타이완달러": "대만 TWD",
    "파운드": "영국 GBP",
    "오마니": "오만 OMR",
    "캐나다달라": "캐나다 CAD",
}

def money_translate(keyword):
    '''150 달라를 입력하면 달라에 대한 환율을 얻어 환율 * 150 의 결과를 리턴하는 함수
    
    Args:
        keyword (str) : 값 통화명 (값과 통화명은 띄어쓰기로 구분)
    
    Returns:
        str : 통화에 따른 환율을 계산하여 문자열로 결과값 리턴
    '''
    
    # get_exchange_info 함수로 현재 환율목록을 새로 구함
    EXCHANGE_LIST = get_exchange_info()
    
    # 입력된 단어를 값과 통화명을 나누어 리스트화 시킵니다. ['150', '달라']
    # 통화명을 반복
    for m in MONEY_NAME.keys():
        # 통화명이 입력된 문자열에 존재한다면
        if m in keyword:
            # 빈 리스트 생성
            keywords = []
            # 문자열에서 통화명이 등장하는 위치까지를 리스트 0번째에 추가
            keywords.append(keyword[0:keyword.find(m)])
            # 1번째 리스트에 통화명 추가 ['150', '달라']
            keywords.append(m)
            break
            
    # 검색어의 통화영역 '달라' 이름이 MONEY_NAME 에 존재하면
    if keywords[1] in MONEY_NAME:
        # '달라' 에 해당하는 '미국 USD' 이름을 구합니다.
        country = MONEY_NAME[keywords[1]]
        
        # '미국 USD' 이름이 네이버 환율 목록에 존재한다면
        if country in EXCHANGE_LIST:
            # 해당 통화에 대한 환율 값을 float() 로 형변환 하여 money 에 저장합니다.
            money = float(EXCHANGE_LIST[country])
            
            # 일본인 경우에는 기준값이 100 이라 100으로 나누어줍니다.
            if country == "일본 JPY (100엔)":
                money /= 100
            
            # 환율 * 사용자입력값 
            money = format(round(float(money) * float(keywords[0]), 3), ",")
            output = "{} 원".format(money)
            return output
    # 검색어의 통화영역 '달라' 이름이 MONEY_NAME 에 없으면 그냥 None 리턴
    return None

print(get_exchange_info())
print(money_translate("150 달라"))
print(money_translate("150달라"))

### /getfile, /getimage.... 등의 로컬 파일 전송 처리

* 텔레그램 API 에는 파일 전송에 대한 API 가 이미 기술되어있습니다.
* telepot 라이브러리에는 sendDocument(), sendPhoto(), sendAudio(), sendVideo() 등의 함수가 텔레그램 API에 맞게 이미 구현되어있습니다.
* 명령(/getfile 등)에 따라 파일을 open하여 함수를 호출 하면 됩니다.
> bot.sendDocument(chat_id, open("파일경로", "rb"))

## 최종 텔레그램 봇을 완성 합니다.

In [ ]:
# 텔레그램봇을 위한 라이브러리
import telepot

# 로그 관련 라이브러리
import logging

# 기능을 분리해놓은 모듈
# modules.py 파일은 같은 경로에 있어야 합니다.
from modules import *

# 로그 생성
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# 봇생성 후 받은 텔레그램 API 토큰
TELEGRAM_TOKEN = "646258683:AAG1Q5_pXsThCvi2J0de_J--plCJpGYDqj0"

def handler(msg):
    '''텔레그램 봇의 메시지 이벤트가 발생되면 콜백호출 되는 함수
    
    Args:
        msg (object) : 텔레그램봇의 데이터가 딕셔너리 형태로 넘어옵니다.
        
    Returns:
        Nothing
    '''
    
    # glance() 함수는 는 telepot 의 텔레그램 메세지객체 에서 헤드라인 정보만 추출해서 리턴하는 함수 입니다.
    content_type, chat_type, chat_id, msg_date, msg_id = telepot.glance(msg, long=True)
    
    # 텍스트 타입인 경우
    if content_type == "text":
        # 채팅으로 입력된 문자열
        str_message = msg["text"]
        # 채팅 시작이 '/' 문자인 경우만 명령어로 인식 합니다.
        if str_message[0:1] == "/":
            # 채팅으로 입력된 명령어에 인자값을 확인하기 위해 공백으로 문자열을 분리 합니다.
            # /money 150달라, /weather 경기도
            args = str_message.split(" ")

            # 리스트의 0번째는 /weather 가 들어가고 1번째는 경기도가 들어갈테니 그걸 구분합니다.
            command = args[0]
            # command 를 구하고 리스트의 0번째를 삭제하면 리스트에는 나머지 인자값만 남습니다.
            # ['경기도']
            del args[0]

            # 명령어에 따른 기능을 수행합니다.
            if command == "/weather" or command == "/날씨":
                # args 는 현재 리스트 형태기 때문에 리스트를 합쳐 하나의 str 변수로 만듭니다.
                # 만약 사용자가 /weather 경 기 도 라고 입력했어도 문제없이 처리 할 수 있습니다.
                args = " ".join(args)
                # modules.py 에 선언된 get_weather 함수를 호출하여 결과를 받습니다.
                weather = get_weather(args)
                # 텔레그램으로 결과를 전송합니다.
                bot.sendMessage(chat_id, weather)
            # 환율 정보 기능 수행
            elif command == "/money" or command == "/환율":
                args = " ".join(args)
                money = money_translate(args)
                bot.sendMessage(chat_id, money)
            # 폴더목록 기능 수행
            # /dir c:\temp 하면 temp 폴더의 목록을 모두 구해 줍니다.
            elif command == "/dir" or command == "/목록":
                args = " ".join(args)
                str_list = get_dir_list(args)
                bot.sendMessage(chat_id, str_list)
            # /getfile, /getimage, /getaudio, /getvidoe 는
            # 공통적인 코드가 많아서 하나로 묶어 처리하고
            # 실제 전송 함수만 다르게 수행합니다.
            elif command[0:4] == "/get":
                filepath = " ".join(args)
                # 파일이 존재하는 경우에만 수행합니다.
                if os.path.exists(filepath):
                    try:
                        # 각 명령어에 따른 함수를 수행합니다.
                        if command == "/getfile":
                            bot.sendDocument(chat_id, open(filepath, "rb"))
                        elif command == "/getimage":
                            bot.sendPhoto(chat_id, open(filepath, "rb"))
                        elif command == "/getaudio":
                            bot.sendAudio(chat_id, open(filepath, "rb"))
                        elif command == "/getvideo":
                            bot.sendVideo(chat_id, open(filepath, "rb"))
                    except Exception as e:
                        bot.sendMessage(chat_id, "파일전송 실패 {}".format(e))
                else:
                    bot.sendMessage(chat_id, "파일이 존재하지 않습니다.\r\n{}".format(filepath))

# telepot 의 객체를 생성하여 bot 변수에 저장합니다.
bot = telepot.Bot(TELEGRAM_TOKEN)

# bot 이 무한반복하며 메세지를 핸들링 합니다.
# 인자값으로 핸들러 함수를 넘기고 run_forever=True 로 종료를 방지합니다.
# 실제 프로그램을 종료 하려면 Ctrl + C 를 눌러 키보드 인터럽트를 발생시킵니다.
bot.message_loop(handler, run_forever=True)